# Creating a SQLite Database Project

In [ ]:
import pandas as pd
import sqlite3

# prevent the DataFrame output from being truncated, given the size of the main game log file:

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [ ]:
# Read the csv files into dataframes so we can explore the data

game_log = pd.read_csv("data/game_log.csv")
park_codes = pd.read_csv("data/park_codes.csv")
person_codes = pd.read_csv("data/person_codes.csv")
team_codes = pd.read_csv("data/team_codes.csv")
appearance_types = pd.read_csv("data/appearance_type.csv")

In [ ]:
# Explore the Game Log data...

game_log.shape
game_log.head(10)
game_log.tail(10)
game_log.describe()

<b>Game Log data: Observations</b>

- There are 171,907 rows of data in this file. There are 161 columns. 
- 'v_' prefix looks to mean 'Visitor'
- 'h_' prefix looks to mean 'Home'
- 1b, 2b and 3b look to mean Forst Base, Second Base and Third Base
- There seems to be a relationship with the park_codes helper file, 'park_id' column.
- There may be a relationship between the columns like * pitcher_id, * player_id and the person_codes.csv eg. 'whitd102'. To check whther this included umpires eg. boakj901.
- There may be a relationship between the v_name and h_name columns and the team_codes helper file data eg. FW1, WS3
- There lots of 'Nan' values
- Theres lots of redundency and opportunities for normalisation!
- There seems to be some very historic data here, with dates starting from 1871

<b>What each defensive position number represents:</b>

There are nine fielding positions in baseball. Each position conventionally has an associated number, which is used to score putouts: 1 (pitcher), 2 (catcher), 3 (first baseman), 4 (second baseman), 5 (third baseman), 6 (shortstop), 7 (left fielder), 8 (center fielder), and 9 (right fielder).

<b> The values in the various league fields, and which leagues they represent:</b>

- Major League Baseball is broken up into two leagues: the American League <b>(AL)</b> & the National League <b>(NL)</b> 
- Federal League <b>(FL)</b> is defunct since 1915, in the United States
- Union Association <b>(UA)</b> was a league in Major League Baseball which lasted for only one season in 1884. 
- Double-A <b>(AA)</b> is the second highest level of play in Minor League Baseball (MiLB) in the United States after Triple-A.
- The Players' League <b>(PL)</b> was a short-lived but star-studded professional American baseball league of the 19th century



In [ ]:
# Explore the Park Codes data...

park_codes.shape
park_codes.head(10)
park_codes.tail(10)
park_codes.describe()

<b>Park Codes data: Observations</b>

- There is 252 parks listed,  9 columns for each
- Links to Game log via 'park_id'
- Contains historic data ie. parks that no longer exist (where there is an end date)

In [ ]:
# Explore the Person Codes data...

person_codes.shape
person_codes.head(10)
person_codes.tail(10)

<b>Person Codes data: Observations</b>

- There is 20,494 people listed, 7 columns for each
- Links to many columns in the Game log via 'id' - a person can be a player, manager, coach and an umpire.

In [ ]:
# Explore the Team Codes data...

team_codes.shape
team_codes.head(10)
team_codes.tail(10)



<b>Team Codes data: Observations</b>

- There is 150 listed, 8 columns for each
- Links to two columns in the Game log via 'team_id'
- Contains historic data, including some teams that are no longer ie. where there is an end date

<b>Creating a Primary Key for Game Log</b>

Check with source of the data (Retrosheet) to see if they already have a system for uniquely identifying each game. 
It means that if at some later stage we choose to incorporate more detailed game data into our database, the keys we use will be compatible with other sources.

Retrosheet has an 'ID' that may be appropriate: Each game begins with a twelve character ID record which identifies the date, home team, and number of the game. 

For example, ATL198304080 should be read as follows:
    - The first three characters identify the home team (the Braves). 
    - The next four are the year (1983). 
    - The next two are the month (April) using the standard numeric notation, 04, followed by the day (08). 
    - The last digit indicates if this is a single game (0), first game (1) or second game (2) if more than one game is played during a day, 
    usually a double header The id record starts the description of a game thus ending the description of the preceding game in the file.

<b>Opportunities to normalize the data</b>

<b>game_log:</b> All of the 'name' fields that have a person_id to associate with person_codes could be removed. This includes Players, Umpires, Managers, Batters
<b>person_codes: </b> Player debut date could be worked out, and possibly ump_debut, but can’t see a mgr_debut or coach_debut opportunity. 
<b>park_codes:</b> Start and end dates could be derived based on the games. ‘League’ is available in team_codes
<b>team_codes:</b> Start and End dates could be derived from the game_log



<b> Normalised Schema Design</b>

Start by creating a diagram of the four existing tables and their columns, and then gradually create new tables that move the data into a more normalized state. Assumptions:

- Historically, teams sometimes move between leagues.
- The same person might be in a single game as both a player and a manager
- Because of how pitchers are represented in the game log, not all pitchers used in a game will be shown. 
- We only want to worry about the pitchers mentioned via position or the 'winning pitcher'/ 'losing pitcher'
- It is possible to over-normalize. We want to finish with about 7-8 tables total.

<b> Proposed Schema for Implementation(Data Quest)</b>
![nlb schema](mlb_schema.svg)

<h3> Implementing a new Schema</h3>

In [120]:
# SQL helper functions from previous exercises 

def run_query(q):   
        with sqlite3.connect('data/mlb.db') as conn:
            return pd.read_sql(q, conn)

def run_command(c):  
    with sqlite3.connect('data/mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)
        
def show_tables():   
    tbl_query = "SELECT name, type FROM sqlite_master WHERE type IN (\"table\",\"view\");"    
    return run_query(tbl_query)

In [121]:
# Read in the data from the csv files into dataframes

game_log = pd.read_csv("data/game_log.csv")
park_codes = pd.read_csv("data/park_codes.csv")
person_codes = pd.read_csv("data/person_codes.csv")
team_codes = pd.read_csv("data/team_codes.csv")
appearance_types = pd.read_csv("data/appearance_type.csv")

# create some working tables in the sqlite db and add the data from the dataframes

with sqlite3.connect('mlb.db') as conn:
    
    game_log.to_sql('game_log', conn, index=False)
    park_codes.to_sql('park_codes', conn, index=False)
    person_codes.to_sql('person_codes', conn, index=False)
    team_codes.to_sql('team_codes', conn, index=False)


In [122]:
# Create a new column in the game_log and add the new ID (as per investigations)

cmd_add_game_id = "ALTER TABLE game_log ADD COLUMN game_id"
cmd_populate_game_id = '''  UPDATE game_log
                            SET game_id = h_name || date || number_of_game
                            '''
run_command(cmd_add_game_id)                    
run_command(cmd_populate_game_id)


In [123]:
# Create the person table 

cmd_create_person = '''CREATE TABLE IF NOT EXISTS person (
                        person_id TEXT PRIMARY KEY,
                        first_name TEXT,
                        last_name TEXT
                    )'''

run_command(cmd_create_person)

In [124]:
# Populate with the person_codes SQL table

cmd_populate_person = '''INSERT OR IGNORE INTO person (
                        person_id, first_name, last_name)
                        SELECT id, first, last FROM person_codes'''

run_command(cmd_populate_person)

In [125]:
# Create the park table

cmd_create_park = '''CREATE TABLE IF NOT EXISTS park (
                        park_id TEXT PRIMARY KEY,
                        name TEXT,
                        nickname TEXT,
                        city TEXT,
                        state TEXT,
                        notes TEXT
                    )'''
run_command(cmd_create_park)

In [126]:
# Populate from the park_codes SQL table 

cmd_populate_park = '''INSERT OR IGNORE INTO park 
                        SELECT park_id, name, aka, city, state, notes FROM park_codes'''

run_command(cmd_populate_park)

In [127]:
# Create the league table 

cmd_create_league = '''CREATE TABLE IF NOT EXISTS league (
                        league_id TEXT PRIMARY KEY,
                        name TEXT
                    )'''


run_command(cmd_create_league)

In [128]:
# Populate based on the above research around the different values

cmd_populate_league = '''INSERT OR IGNORE INTO league 
                        VALUES ("AL", "American League"),
                                ("NL", "National League"), 
                                ("FL", "Federal League"), 
                                ("UA", "Union Association"), 
                                ("AA", "Double A"), 
                                ("PL", "Players League")'''
run_command(cmd_populate_league)

In [129]:
# Create the appearance_type table 

cmd_create_appearance_type = '''CREATE TABLE IF NOT EXISTS appearance_type (
                        appearance_type_id TEXT PRIMARY KEY,
                        name TEXT,
                        category TEXT
                    )'''

run_command(cmd_create_appearance_type)

In [130]:
# Populate from appearance_type dataframe

appearance_types.to_sql('appearance_type', conn, index=False, if_exists='append')


In [131]:
# Create the game table 

cmd_create_game = '''CREATE TABLE IF NOT EXISTS game (
                    game_id TEXT PRIMARY KEY,
                    date DATETIME,
                    number_of_game INTEGER,
                    park_id TEXT,
                    length_outs REAL,
                    day BOOLEAN,
                    completion TEXT,
                    forfeit TEXT,
                    protest TEXT,
                    attendance REAL,
                    length_minutes REAL,
                    additional_info TEXT,
                    acquisition_info TEXT,
                    FOREIGN KEY (park_id) REFERENCES park(park_id)                    
                    )'''
run_command(cmd_create_game)

In [132]:
# Populate from the game_log SQL Table

cmd_populate_game = '''INSERT OR IGNORE INTO game 
                        SELECT game_id, 
                                date, 
                                number_of_game, 
                                park_id,  
                                length_outs, 
                                day_night, 
                                completion, 
                                forefeit,
                                protest,
                                attendance,
                                length_minutes,
                                additional_info,
                                acquisition_info
                        FROM game_log'''

run_command(cmd_populate_game)

In [133]:
# Create the team table

cmd_create_team = '''CREATE TABLE IF NOT EXISTS team (
                    team_id TEXT PRIMARY KEY,
                    league_id TEXT,
                    city TEXT,
                    nickname TEXT,
                    franch_id TEXT,
                    FOREIGN KEY (league_id) REFERENCES league(league_id)
                    )'''

run_command(cmd_create_team)

In [134]:
# Populate the team table from the team_codes table

cmd_populate_team = '''INSERT OR IGNORE INTO team
                        SELECT team_id,
                                league,
                                city,
                                nickname,
                                franch_id
                        FROM team_codes'''

run_command(cmd_populate_team)

In [140]:
# Create the team_appearance table

cmd_create_team_appearance = ''' CREATE TABLE IF NOT EXISTS team_appearance (
                                "team_id" TEXT,
                                "game_id" TEXT,
                                "home" BOOLEAN,
                                "league_id" TEXT,
                                "score" INTEGER,
                                "line_score" TEXT,
                                "at_bats" REAL,
                                "hits" REAL,
                                "doubles" REAL,
                                "triples" REAL,
                                "homeruns" REAL,
                                "rbi" REAL,
                                "sacrifice_hits" REAL,
                                "sacrifice_flies" REAL,
                                "hit_by_pitch" REAL,
                                "walks" REAL,
                                "intentional_walks" REAL,
                                "strikeouts" REAL,
                                "stolen_bases" REAL,
                                "caught_stealing" REAL,
                                "grounded_into_double" REAL,
                                "first_catcher_interference" REAL,
                                "left_on_base" REAL,
                                "pitchers_used" REAL,
                                "individual_earned_runs" REAL,
                                "team_earned_runs" REAL,
                                "wild_pitches" REAL,
                                "balks" REAL,
                                "putouts" REAL,
                                "assists" REAL,
                                "errors" REAL,
                                "passed_balls" REAL,
                                "double_plays" REAL,
                                 "triple_plays" REAL,
                                 PRIMARY KEY (team_id, game_id),
                                 FOREIGN KEY (league_id) REFERENCES league(league_id),
                                 FOREIGN KEY (team_id) REFERENCES team(team_id),
                                 FOREIGN KEY (game_id) REFERENCES game(game_id)
                                )'''

run_command(cmd_create_team_appearance)

In [141]:
# Populate the team appearance table, based on the fields in game_log for each home team and visitor team

cmd_populate_team_appearance='''INSERT OR IGNORE INTO team_appearance
    SELECT v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays   
  FROM game_log
UNION
   SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays   
  FROM game_log  '''

run_command(cmd_populate_team_appearance)

In [142]:
# Create table person_appearance

cmd_create_person_appearance = '''CREATE TABLE IF NOT EXISTS person_appearance (
                                appearance_id INTEGER PRIMARY KEY,
                                person_id TEXT,
                                team_id TEXT,
                                game_id TEXT,
                                appearance_type_id TEXT,
                                FOREIGN KEY (person_id) REFERENCES person(person_id),
                                FOREIGN KEY (team_id) REFERENCES team(team_id),
                                FOREIGN KEY (game_id) REFERENCES game(game_id),
                                FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
)'''
run_command(cmd_create_person_appearance)

In [147]:
# Populate the person_appearance_table with the values for the umpire fields in game_log
# ie. hp_umpire_id, 1b_umpire_id, 2b_umpire_id, 3b_umpire_id, lf_umpire_id, rf_umpire_id

cmd_populate_umpires = '''
                        INSERT OR IGNORE into person_appearance
                            (
                            person_id,
                            team_id,
                            game_id,
                            appearance_type_id
                            )
                        SELECT 
                            hp_umpire_id,
                            NULL,
                            game_id,
                            'UHP'
                        FROM game_log
                        WHERE hp_umpire_id IS NOT NULL
                    UNION    
                        SELECT 
                            [1b_umpire_id],
                            NULL,
                            game_id,
                            'U1B'
                        FROM game_log
                        WHERE [1b_umpire_id] IS NOT NULL
                    UNION     
                        SELECT
                            [2b_umpire_id],
                            NULL,
                            game_id,
                            'U2B'
                        FROM game_log
                        WHERE [2b_umpire_id] IS NOT NULL
                    UNION
                        SELECT
                            [3b_umpire_id],
                            NULL,
                            game_id,
                            'U3B'
                        FROM game_log
                        WHERE [3b_umpire_id] IS NOT NULL
                    UNION
                        SELECT
                            lf_umpire_id,
                            NULL,
                            game_id,
                            'ULF'
                        FROM game_log
                        WHERE lf_umpire_id IS NOT NULL
                    UNION
                        SELECT
                            rf_umpire_id,
                            NULL,
                            game_id,
                            'URF'
                        FROM game_log
                        WHERE rf_umpire_id IS NOT NULL
                        '''
run_command(cmd_populate_umpires)

In [149]:
# Populate the person_appearance_table with the values for the manager fields in game_log
# ie. v_manager_id, h_manager_id

cmd_populate_managers = '''
                        INSERT OR IGNORE into person_appearance
                            (
                            person_id,
                            team_id,
                            game_id,
                            appearance_type_id
                            )
                        SELECT
                            v_manager_id,
                            v_name,
                            game_id,
                            "MM"
                        FROM game_log
                        WHERE v_manager_id IS NOT NULL
                    UNION
                        SELECT
                            h_manager_id,
                            h_name,
                            game_id,
                            "MM"
                        FROM game_log
                        WHERE h_manager_id IS NOT NULL'''

run_command(cmd_populate_managers)


In [150]:
# Populate person_appearance with the values for the pitcher fields in game_log 
# ie. winning_pitcher_id, losing_pitcher_id, saving_pitcher_id, winning_rbi_batter_id, v_starting_pitcher_id, h_starting_pitcher_id

cmd_populate_pitchers_batters = '''INSERT OR IGNORE INTO person_appearance
                             (
                            person_id,
                            team_id,
                            game_id,
                            appearance_type_id
                            )
                            SELECT
                                winning_pitcher_id,
                                CASE
                                    WHEN h_score > v_score THEN h_name
                                    ELSE v_name
                                END,
                                game_id,
                                "AWP"
                            FROM game_log
                            WHERE winning_pitcher_id IS NOT NULL
                        UNION
                            SELECT
                                losing_pitcher_id,
                                CASE
                                    WHEN h_score > v_score THEN v_name
                                    ELSE h_name
                                END,
                                game_id,
                                "ALP"
                                FROM game_log
                                WHERE losing_pitcher_id IS NOT NULL
                        UNION
                            SELECT
                                saving_pitcher_id,
                                NULL,
                                game_id,
                                "ASP"
                            FROM game_log
                            WHERE saving_pitcher_id IS NOT NULL
 
                        UNION
                            SELECT
                                winning_rbi_batter_id,
                                CASE
                                    WHEN h_score > v_score THEN h_name
                                    ELSE v_name
                                    END,
                                game_id,
                                "AWB"
                            FROM game_log
                            WHERE winning_rbi_batter_id IS NOT NULL
                        UNION
                            SELECT
                                v_starting_pitcher_id,
                                v_name
                                game_id,
                                "PSP"
                            FROM game_log
                            WHERE v_starting_pitcher_id IS NOT NULL
                        UNION
                            SELECT
                                h_starting_pitcher_id,
                                h_name
                                game_id,
                                "PSP"
                            FROM game_log
                            WHERE h_starting_pitcher_id IS NOT NULL
                            '''

run_command(cmd_populate_pitchers_batters)

In [151]:
# Populate person_appearance with the values for the player fields in game_log, 
# we essentially are performing 36 permutations: 2 (home, visitors) * 2 (offense + defense) * 9 (9 positions) so use loops

cmd_populate_players = '''INSERT OR IGNORE INTO person_appearance (
                            game_id,
                            team_id,
                            person_id,
                            appearance_type_id
                            ) 
                        SELECT
                            game_id,
                            {hv}_name,
                            {hv}_player_{num}_id,
                            "O{num}"
                        FROM game_log
                        WHERE {hv}_player_{num}_id IS NOT NULL

                    UNION

                        SELECT
                            game_id,
                            {hv}_name,
                            {hv}_player_{num}_id,
                            "D" || CAST({hv}_player_{num}_def_pos AS INT)
                        FROM game_log
                        WHERE {hv}_player_{num}_id IS NOT NULL;
                        '''
for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(cmd_populate_players.format(**query_vars))
    

In [1]:
# clean up the temmporary working tables in the SQLite db

run_command("DROP TABLE game_log")
run_command("DROP TABLE park_codes")
run_command("DROP TABLE team_codes")
run_command("DROP TABLE person_codes")